In [2]:
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI
client = OpenAI()

class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))

assistant = client.beta.assistants.create(
  name="Book Assistant",
  instructions="You help users with their question on the files they upload",
  tools=[{"type" : "code_interpreter", "type" : "file_search"}],
  model="gpt-4o",
)


assistant_id = "asst_So5uPjPWR61Gy3wOf2x853ZD"



Assistant(id='asst_So5uPjPWR61Gy3wOf2x853ZD', created_at=1717907719, description=None, instructions='You help users with their question on the files they upload', metadata={}, model='gpt-4o', name='Book Assistant', object='assistant', tools=[FileSearchTool(type='file_search', file_search=None)], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=1.0)

In [5]:
vector_store = client.beta.vector_stores.create(name="Financial Statements")
 
# Ready the files for upload to OpenAI
file_paths = ["files/novel.txt"]
file_streams = [open(path, "rb") for path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [6]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [7]:
message_file = client.files.create(
  file=open("files/novel.txt", "rb"), purpose="assistants"
)


thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Summarize the novel",
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_x4h1WyAzInZmguPmsv5Ssg8D'])


In [4]:
file = client.files.create(file=c   .file_from_path("./files/novel.txt"), purpose="assistants")
client.files.upload(file.id, "./files
file

AttributeError: 'OpenAI' object has no attribute 'file_from_path'

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

In [ ]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )

def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function : {function.name} with arg {function.arguments}")
        outputs.append({
            "output" : functions_map[function.name](json.loads(function.arguments)),
            "tool_call_id" : action_id
        })
    return outputs

def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id = run_id,
        thread_id = thread_id,
        tool_outputs = outputs
    )

In [ ]:
#get_run(run.id, thread.id).status
#get_run(run.id, thread.id).status
submit_tool_outputs(run.id, thread.id)
#send_message(thread.id, "Please go ahead!")

In [ ]:
get_run(run.id, thread.id).status

#get_messages(thread.id)

In [ ]:
send_message(thread.id, "Now I want to know if Cloudflare is a good buy.")

In [ ]:
get_messages(thread.id)